In [151]:
import math
import numpy as np
from scipy.stats import mode
import sys
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
from helper import *
import pandas as pd
from google.colab import files
import io

print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.10.12


In [152]:
from google.colab import drive
drive.mount('/content/drive')#https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
df = pd.read_csv('/content/KNNAlgorithmDataset.csv')
df['diagnosis'] = df['diagnosis'].replace({'M': int('0')})
df['diagnosis'] = df['diagnosis'].replace({'B': int('1')})
df.shape

(569, 33)

In [154]:
Xrawdata, Yrawdata = df.shape
trpercent = 0.8

In [155]:
dfxTr = df.iloc[:math.floor(Xrawdata*trpercent),2:32]
dfyTr = df.iloc[:math.floor(Xrawdata*trpercent),1:2]
dfxTe = df.iloc[math.floor(Xrawdata*trpercent):Xrawdata,2:32]
dfyTe = df.iloc[math.floor(Xrawdata*trpercent):Xrawdata,1:2]

In [156]:
xTr = dfxTr.to_numpy()
yTr = dfyTr.to_numpy()
xTe = dfxTe.to_numpy()
yTe = dfyTr.to_numpy()

In [157]:
def innerproduct(X,Z=None):
    if Z is None:
        return X.dot(X.T)
    else:
        return X.dot(Z.T)

In [158]:
def l2distance(X,Z=None):
    if Z is None:
        n,d = X.shape
        m = n
        R = np.broadcast_to(innerproduct(X).diagonal(),(n,m))
        S = R.T
        G=innerproduct(X,Z)
        D =np.sqrt(S + R - (2*G))

    else:
        n,d = X.shape
        m= Z.shape[0]
        S=np.broadcast_to(innerproduct(X).diagonal(),(m,n)).T
        R=np.broadcast_to(innerproduct(Z).diagonal(),(n,m))
        G=innerproduct(X,Z)
        D =np.sqrt(S + R - (2*G))
    return D

In [159]:
def findknn(xTr,xTe,k):
    Distance = l2distance(xTr,xTe)
    indices=np.argsort(Distance,axis=0)[:k]
    dists=np.sort(Distance,axis=0)[:k]

    return indices, dists

In [160]:
def accuracy(truth,preds):

    truth = truth.flatten()
    preds = preds.flatten()
    diff = np.abs(truth - preds)
    false = np.count_nonzero(diff)
    corr = diff.shape[0] - false
    accuracy = np.float64(corr / diff.shape[0])

    return accuracy

In [161]:
def knnclassifier(xTr,yTr,xTe,k):

    yTr = yTr.flatten()

    indices, dists = findknn(xTr,xTe,k)
    label = yTr[indices]
    preds = mode(label)[0][0]
    return preds

In [162]:
preds = knnclassifier(xTr,yTr,xTe,1)
result=accuracy(yTe,preds)
t1 = time.time()
print("You obtained %.2f%% classification acccuracy in %.4f seconds\n" % (result*100.0,t1-t0))

You obtained 59.12% classification acccuracy in 737.5873 seconds

